# Bankfull characteristics model: Spatiotemporal evolution of bankfull discharge and channel geometry of an alluvial meandering river. Part 1: Fundamental behavior of the model.

Kensuke Naito (knaito2@illinois.edu) and Gary Parker

This is a numerical model presented in a paper titled "Can Bankfull Discharge and Bankfull Channel Characteristics of an Alluvial River be Co-specified from the Flow Duration Curve? Part 1: Framework for Analysis", which is submitted to Journal of Geophydical Research: Earth Surface in December 2018. The model describes spatiotemporal variation of the bankfull characteristics (bankfull discharge (Qbf), bankfull width (Bbf), bankfull depth (Hbf) and down-channel slope (Sc)) for specified flow duration curve, annual bed material feed rate and other input parameters. From arbitrarily selected initial condition, the model ddepicts the evolution of bankfull characteristics to an equilibrium state, where the variables show no further change both in space and time. 

Here, fundamental behavior of the model for the case of the Trinity River, TX, USA is presented. 

The script is written in Python 2.7. However, it is made in such a way that the model can also be run with PyPy, which is fast, compliant alterative implementation of Python. PyPy can be downloaded at https://pypy.org/.

## 1. Model setup

#### Import libraries

In [ ]:
import os 
import sys
import numpy as np
import csv  

#### Set path and file name

Set path to working directory and the folder where model output csv files will be saved. The file names of the flow duration curve as well as output file need to be specified here.

In [ ]:
path_work = 'path to your working directory where fdc files are stored'
path_csv =  'path to directory where you wish to store csv files'
fdc_filename = '_fdc_trinityriver_romayor.csv'
out_filename = '_yr_results'

#### Input parameters

Most of the input parameters are determined based on USGS gaging station at the Trinity River near Romayor, TX, USA. USGS gaging station number: 08066500.

In [ ]:
# Space / time grid size
dxv = 1000.          # [m] Spatial grid size (valley length)
M = 21          # [1] Number of spatial grids
dt = 0.001          # [yr]  Temporal grid size 
Nloop = 3000001          # [1] Number of itteratoion for the spin-up run
Ntoprint = 100000          # Number of itteration untill next printout for the spin-up run
au = 0.5          # [1] au = 0.5: Central difference, au = 1: Full upwind

# Initial channel geometry
BbfI = 220.          # [m] Bankfull width   
HncI = 2.          # [m] Thickness of lower floodplain layer
HcI = 1.          #  [m] Thickness of upper floodplain layer
HbfI = HncI + HcI          # [m] Bankfull deoth
ScI = 0.00008          # [1] Down-channel slope
Etab_d = 10.          # [m] Channel bed elevation at the downstream end 

# Parameter for the flow duration curve
Nfdc = 100          #[1] Number of bins in FDC 

# Calibration parameter
aEH = 0.45          # [1] Engelund-Hansen calibration factor
Tsb = 0.1          # [yr] Characteristic bank armor (slump block) residence time
cveg = 8.          # [m/yr] Characteristic vegetal encroachment rate
fe = 0.1          # [1] Fractional efficiency of floodplain sedimentation

# Auxullary parameter
Cz = 14.          # [1] Dimensionless Chezy channel resistance coeffieent

# Sediment and channel planform
Dbm = 0.35 / 1000.         # [mm] D50 of bed material
QTfeed = 0.22          # [Mt/yr] Mean annual bed material feed rate
Dfm = 0.04 / 1000.          # [mm(m)] floodplain material size (0.05)
Cfm = 2e-5          # [1] Characteristic volumetric concentration of floodplain material
sinu = 1.8          # [1] Channel sinuosity

 # Parameter for cut bank erosion and inner bank migration
A = 4.81          # [1] Dimensionless coeff (Johannesson & Parker, 1989)
fmb = 10.          # [1] Ratio of meandering belt width to bankfull width (10)
fc = 1.          # [1] Ratio of slump block size to cohesive layer thickness (0.5)
lpc = 0.3          # [1] Porosity of upper floodplain layer
lp = 0.3          # [1] Porosity of channel bed and lower floodplain layer

# Global constants
g = 9.81          # [m/s^2] Gravitational acceleration
Rr = 1.65          # [1] Submerged specific gravity of sediment
day_in_sec = 60. * 60. * 24.          #[s] A day in seconds
year_in_sec = day_in_sec * 365.25          #[s] A year in seconds 
converter_QT = 1e6 / (Rr + 1.)          # [] Unit converter for bed mateiral load

# Unit conversion
Xv = np.ones(M)          # River valley coordinates for plotting results
for j in range(M):
    Xv[j] *= j * dxv / 1000.
QTfeed *= converter_QT          # [Mt/yr -> m^3/yr] Bed mateiral load

# Fall velocity of floodplain material (Dietrich, 1982)
Rep = np.sqrt(g * Rr * Dfm**3.) / 1e-6          # [1] Particle Reynolds number
b1 = 2.89139447769084; b2 = 0.95296; b3 = 0.0568346711984055 
b4 = 0.00289204602084475; b5 = 0.00024464688411386
Rf = np.exp(-b1 + b2 * np.log(Rep) - b3 * np.log(Rep)**2. - b4 * np.log(Rep)**3. + b5 * np.log(Rep)**4.)
vfall = Rf * np.sqrt(g * Rr * Dfm)          # [m/s] Fall velocity of floodplain material 

# Coefficients for point bar height calculation and cut bank erosion rate calculation
fpb = (sinu - 1.) * A / fmb          # [1] For point bar height calculation
fbe = fc / ((1. - lpc) * Tsb)          # [1/yr] For outer bankf erosion calculation

## 2. Main routine 

This rutine includes initial condition setup, computation of the spatial and temporal bankfull characteristics evolution and result output to csv files.

In [ ]:
def main():
    
    # Display total number of years of the calculation and time step between result outputs
    print('Total run [yr] = ' + str(int(dt * Nloop)))
    print('Print out step [yr] = ' + str(int(dt * Ntoprint)))
    
    # Change current directory to working directory
    os.chdir(path_work)

    # Initialize the time
    Time = 0.

    # Constract FDC from csv file on daily-averge flow record
    Qi, pi = construct_fdc(fdc_filename)

    # Calculate and setup initial condition
    usbf = (g * HbfI * ScI)**(1./2.)          # [m/s] Bankfull shear velocity
    QbfI = Cz * usbf * HbfI * BbfI          # [m^3/s] Bankfull discharge
    HpbI = fpb * HbfI          # [m] Point bar height
    cEI = outerbank_erosion_rate(QbfI, BbfI, HbfI, HncI, ScI, Qi, pi)          # [m/yr] Cut bank erosion rate
    cDI = innerbank_deposition_rate(QbfI, BbfI, HbfI, ScI, Qi, pi)          # [m/yr] Inner bank deposition rate
    vDI = overbank_deposition_rate(QbfI, Qi, pi)          # [m/yr] Overbank deposition rate
    QTI = bedmaterial_load(QbfI, BbfI, HbfI, ScI, Qi, pi)          # [m^3/yr] Annual bed material load

    # Set up the spatial grids for initial condition
    Qbf = QbfI * np.ones(M); Bbf = BbfI * np.ones(M)
    Hbf = HbfI * np.ones(M); Sc = ScI * np.ones(M)
    Hnc = HncI * np.ones(M); Hc = HcI * np.ones(M)
    Hpb = HpbI * np.ones(M)
    Etab = Etab_d * np.ones(M)
    for j in range(M):
        Etab[j] += ScI * sinu * dxv * (M - j - 1.)
    Etanc = Etab + HncI; Etac = Etab + HbfI
    Etapb = Etab + HpbI
    cE = cEI * np.ones(M); cD = cDI * np.ones(M)
    vD = vDI * np.ones(M); QT = QTI * np.ones(M)

    # Print out initial condition to csv file
    printout_csv(out_filename, Time, Xv, Qbf, Bbf, Hbf, Sc, Hc, Hnc, Hpb,
                 Etab, Etac, Etanc, Etapb, cE, cD, vD, QT)

    # Temporal loop
    for k in range(Nloop):

        # Advance the clock
        Time += dt
        
        # Spatial evolution of the variables
        Stop, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
              Etab, Etac, Etanc, Etapb, cE, cD, vD, QT  \
              = spatial_calculation(Qbf, Bbf, Sc, Hnc, Hpb,
                                    Etac, Etanc, Etab, Etapb,
                                    cE, cD, vD, QT, QTfeed, Qi, pi)

        # Detect error and print out results
        if Stop == True:
            break
        elif k != 0 and k % Ntoprint == 0:
            printout_csv(out_filename, Time, Xv, Qbf, Bbf, Hbf, Sc, Hc, Hnc, Hpb,
                         Etab, Etac, Etanc, Etapb, cE, cD, vD, QT)
            print(str(int(100 * k / Nloop)) + '% Done')
    
    # Display the main results
    if Stop == False:
        print('===== MAIN RESULTS =====')
        print('Qbf [m^3/s] = ' + str(round(Qbf[0], 1)))
        print('Bbf [m] = ' + str(round(Bbf[0], 1)))
        print('Hbf [m] = ' + str(round(Hbf[0], 1)))
        print('Sc [1] = ' + str(round(Sc[0], 7)))

## 3. Sub-functions

#### Spatial variation of the bankfull characteristics 

Bankfull channel characteristics are calculated as a result of floodplain thickening due to overbank deposition or/and channel bed degradation, floodplain thinning due to cut bank erosion or/and channel bed aggracation, channel widening/narrowing due to the difference in cut bank migraion rate and inner bank migration rate. 

In [ ]:
def spatial_calculation(Qbf, Bbf, Sc, Hnc, Hpb, Etac, Etanc, Etab, Etapb,
                        cE, cD, vD, QT, Qtfeed, Qi, pi):

    # Create arrays
    d_Etab = np.zeros(M); d_Etanc = np.zeros(M)
    d_Etac = np.zeros(M); d_Bbf = np.zeros(M)

    # Calculate update
    for j in range(M):
        # QT at adjacent nodes
        QTmid = QT[j]
        if j == 0:
            QTup = QTfeed
            QTdw = QT[j+1]
        elif j == M-1:
            QTup = QT[j-1]
            QTdw = QTmid
        else:
            QTup = QT[j-1]
            QTdw = QT[j+1]

        d_Etab[j] = (Hnc[j] * cE[j] - Hpb[j] * cD[j]) / Bbf[j] \
                    - (au * (QTmid - QTup) + (1. - au) * (QTdw - QTmid)) \
                    / (dxv * sinu * (1. - lp) * Bbf[j])
        d_Etanc[j] = (Etapb[j] - Etanc[j]) * sinu * cD[j] / (fmb * Bbf[j] - sinu * Bbf[j])
        d_Etac[j] = vD[j] / (1. - lpc) + sinu * (cD[j] * (Etapb[j] - Etanc[j]) - cE[j] * (Etac[j] - Etanc[j])) \
                    / (fmb * Bbf[j] - sinu * Bbf[j])
        d_Bbf[j] = cE[j] - cD[j]

    # Update geometry
    Bbf = Bbf + dt * d_Bbf
    Etab = Etab + dt * d_Etab
    Etanc = Etanc + dt * d_Etanc
    Etac = Etac + dt * d_Etac
    Hnc = Etanc - Etab
    Hc = Etac - Etanc
    Hbf = Hnc + Hc
    Hpb = fpb * Hbf
    Etapb = Etab + Hpb
        
    # Calculate bankfull discharge, slope, channel migration, 
    # overbank deposition and bed material load
    for j in range(M):
        if j == 0:
            Sc[j] = (Etab[0] - Etab[1]) / (sinu * dxv)
        elif j == M - 1:
            Sc[j] = (Etab[-2] - Etab[-1]) / (sinu * dxv)
        else:
            Sc[j] = (Etab[j-1] - Etab[j+1]) / (2. * sinu * dxv)

        # Detect NAN and negative geometry
        if Bbf[j] != Bbf[j] or QT[j] != QT[j] or Bbf[j] <= 0 or Sc[j] <= 0:
            Stop = True
            print '[!] Encountered NAN or negative geometry. Change dt, dx or initial condition'
            break
        else:
            Stop = False
            # Bankfull discharge
            usbf = (g * Hbf[j] * Sc[j])**(1./2.)
            Qbf[j] = Cz * usbf * Hbf[j] * Bbf[j]
            # Lateral channel migration, overbank deposition and bed mateiral load
            cE[j] = outerbank_erosion_rate(Qbf[j], Bbf[j], Hbf[j], Hnc[j], Sc[j], Qi, pi)  # [m/yr]
            cD[j] = innerbank_deposition_rate(Qbf[j], Bbf[j], Hbf[j], Sc[j], Qi, pi)  # [m/yr]
            vD[j] = overbank_deposition_rate(Qbf[j], Qi, pi)  # [m/yr]
            QT[j] = bedmaterial_load(Qbf[j], Bbf[j], Hbf[j], Sc[j], Qi, pi)  # [m^3/yr]
    
    return Stop, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
           Etab, Etac, Etanc, Etapb, cE, cD, vD, QT

#### Function to calculate inner bank depositional migration

Inner bank depositional migration rate (cD) is calculated as a negative function of specified characteristic point bar vegetal encroachment rate (cveg) and bed shear stress (taus). Note that cD is integrated over the specified flow duration curve.  

In [ ]:
def innerbank_deposition_rate(Qbf, Bbf, Hbf, Sc, Qi, pi):
    tausbf = Hbf * Sc / (Rr * Dbm)
    cD = 0.
    for i in range(Nfdc):
        # No inner bank advance during overbank flows
        if Qi[i] <= Qbf:
            H = (Qi[i]**2. / (Cz**2. * g * Sc * Bbf**2.))**(1./3.)
            tausi = H * Sc / (Rr * Dbm)
            cD += 0.5 * cveg / (1. - lp) * (1. - tausi / tausbf) * pi[i]
    return cD

#### Cut bank erosional migration 

Outer bank erosional migration rate (cE) is calculated as a function of specified characteristic bank aromor (slump block) residence time (Tsb) and bed shear stress (taus). Note that cE is integrated over the specified flow duration curve.  

In [ ]:
def outerbank_erosion_rate(Qbf, Bbf, Hbf, Hnc, Sc, Qi, pi):
    if Hnc < 0.:
        Hnc = 0.
    tausbf = Hbf * Sc / (Rr * Dbm)
    cE = 0.
    for i in range(Nfdc):
        # Below bankfull flow
        if Qi[i] <= Qbf:
            H = (Qi[i]**2. / (Cz**2. * g * Sc * Bbf**2.))**(1./3.)
            tausi = H * Sc / (Rr * Dbm)
            cE += 0.5 * fbe * Hnc * (tausi / tausbf) * pi[i]
        # Above bankfull flow
        else:
            cE += 0.5 * fbe * Hnc * 1. * pi[i]
    return cE

#### Overbank deposition 

Overbank deposition rate (vD) is calculated as a function of fall velocity of specified fractional efficiency of floodplain mateiral sedimentation (fe), specified characteristic volume concentration of floodplain mateiral during overbank flows (Cfm) and settling velocity of specified floodplain mateiral grain size (vfall). Note that vD is integrated over the specified flow duration curve.  

In [ ]:
def overbank_deposition_rate(Qbf, Qi, pi):
    vdep = 0.
    for i in range(Nfdc):
        # No overbank deposition rate during below-bankfull flows
        if Qi[i] > Qbf:
            vdep += fe * vfall * year_in_sec * Cfm * pi[i]
    return vdep

#### In-channel bed mateiral transport

In-channel bedmaterial load (QT) is calculated using total load relation by Engelund and Hansen (1967). It is calculated as a function of specified total load relation calibration factor (aEH), bed shear stress (taus) and specified dimensionless Chezy channel resistance coefficieit (Cz). Note that QT is integrated over the specified flow duration curve.

In [ ]:
def bedmaterial_load(Qbf, Bbf, Hbf, Sc, Qi, pi):
    tausbf = Hbf * Sc / (Rr * Dbm)
    Einstein_no = 0.
    for i in range(Nfdc):
        # Below overbank flow
        if Qi[i] <= Qbf:
            H = (Qi[i]**2. / (Cz**2. * g * Sc * Bbf**2.))**(1./3.)
            tausi = H * Sc / (Rr * Dbm)
            Einstein_no += Bbf * aEH * 0.05 * Cz**2. * tausi**(5./2.) * pi[i]
        # Above overbank flow
        else:
            Einstein_no += Bbf * aEH * 0.05 * Cz**2. * tausbf**(5./2.) * pi[i]
    QT = Einstein_no * (g * Rr * Dbm**3.)**(1./2.) * year_in_sec          # [m^3/yr]
    return QT

#### Read csv file to condtruct the flow duration curve

The cvs file containes 4 columns (1: agency, 2: site no, 3: date, 4: discharge). Discharge is provided in US unit, thus it is converted to SI unit. 

In [ ]:
def construct_fdc(fdc_filename):
    # Read csv file
    with open(fdc_filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        Qw = []; counter = 0
        for row in csv_reader:
            if str(row[3]).isdigit() and str(row[3]) > 0.:
                Qw.append(float(row[3]) * (12.**3.) * (2.54**3.) / (100.**3.))     #[ft^3/s -> m^3/s]
            counter += 1
    Qw = np.array(Qw)
    # Condtruct probability density function
    Qi, pi = np.zeros(Nfdc), np.zeros(Nfdc)
    nn, bins = np.histogram(Qw, Nfdc, density=True)
    for k in range(Nfdc):
        Qi[k] = 0.5 * (bins[k] + bins[k+1])
        pi[k] = nn[k] * (bins[k+1] - bins[k])
    return Qi, pi

#### Print out results to csv files

Each output file containes spatial variation of valley coordinate (Xv), bankfull discharge (Qbf), bankfull width (Bbf), bankful depth (Hbf = Hnc + Hc)), down-channel slope (Sc), elevation of channel bed (Etab), elevation of top of upper cohesive floodplain layer (Etac), elevation of top of lower non-cohesive floodplain layer (Etanc), elevation of top of point bar (Etapb), thichness of upper cohesive floodplain layer (Hc = Etac - Etanc), thickness of lower non-cohesive floodplain layer (Hnc = Etanc - Etab), point bar height (Hpb = Etapb - Etab), cut bank erosional migraion rate (cE), inner bankf depositional migraion rate (cD), overbank deposition rage (vD) and annual bed material load (QT). 

In [ ]:
def printout_csv(out_filename, Time, Xv, Qbf, Bbf, Hbf, Sc, Hc, Hnc, Hpb,
                 Etab, Etac, Etanc, Etapb, cE, cD, vD, QT):
    # Change the directory
    os.chdir(path_csv)
    Nrow = len(Xv)
    QT /= converter_QT     # [m^3/yr -> Mt/yr]
    with open(str(int(Time))+out_filename+'.csv', 'wb') as ff:
        writer = csv.writer(ff)
        for kk in range(Nrow):
            # [0: Xv, 1:Qbf, 2:Bbf, 3:Hbf, 4:Sc, 5:Etab, 6:Etac, 7:Etanc, 8:Etapb,
            #  9:Hc, 10:Hnc, 11:Hpb, 12:cE, 13:cD, 14:vD, 15:QT]
            writer.writerow([
                '{:.1f}'.format(Xv[kk]), '{:.2f}'.format(Qbf[kk]), '{:.2f}'.format(Bbf[kk]),
                '{:.2f}'.format(Hbf[kk]), '{:.7f}'.format(Sc[kk]),
                '{:.2f}'.format(Etab[kk]), '{:.2f}'.format(Etac[kk]), '{:.2f}'.format(Etanc[kk]),
                '{:.2f}'.format(Etapb[kk]), '{:.2f}'.format(Hc[kk]), '{:.2f}'.format(Hnc[kk]),
                '{:.2f}'.format(Hpb[kk]), '{:.2f}'.format(cE[kk]), '{:.2f}'.format(cD[kk]),
                '{:.7f}'.format(vD[kk]), '{:.4f}'.format(QT[kk])
                ])
    # Change the directory back
    os.chdir(path_work)

## 4. Run the script

In [ ]:
if __name__ == '__main__':
    main()